# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()
DATA_DIR = ROOT_DIR / "data"

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = DATA_DIR / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# DocumentImageClassifier 를 사용해보기

In [4]:
from src.model.classifier import DocumentImageClassifier

In [5]:
SEED = 432
MODEL_NAME = "resnet34"
LEARNING_RATE = 1e-3
IMAGE_SIZE = 256
BATCH_SIZE = 64
VAL_RATE = 0.2
NUM_WORKERS = 4
EPOCHS = 10

In [6]:
DocumentImageClassifier(
    model_name=MODEL_NAME,
    num_classes=17,
    learning_rate=LEARNING_RATE,
)

DocumentImageClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

## PyTorch Lightning의 Trainer와 함께 사용해보기

### Trainer 매개변수 설명

#### 🔴 필수 설정 (반드시 고려해야 함)
- **max_epochs** (기본값: `None`)
  - 총 학습할 에포크 수를 설정합니다.
  - 기본값이 `None`이므로 무한 학습됩니다. 실제로는 필수 설정입니다.

#### 🟡 권장 설정 (기본값이 있지만 명시 권장)
- **accelerator** (기본값: `"auto"`)
  - 사용할 하드웨어 가속기를 지정합니다.
  - `"auto"`: 자동 선택
  - `"cpu"`: CPU 사용
  - `"gpu"`: GPU 사용 (CUDA)
  - `"mps"`: Apple Silicon GPU 사용

- **devices** (기본값: `"auto"`)
  - 사용할 디바이스 수나 특정 디바이스를 지정합니다.
  - `"auto"`: 사용 가능한 모든 디바이스
  - `1`, `2`: 디바이스 개수
  - `[0, 1]`: 특정 디바이스 ID

- **precision** (기본값: `"32-true"`)
  - 연산 정밀도를 설정하여 메모리 사용량과 속도를 최적화합니다.
  - `"32"`: 32비트 부동소수점 (기본값)
  - `"16-mixed"`: 16비트 혼합 정밀도 (AMP) - 메모리 절약 권장
  - `"bf16-mixed"`: BFloat16 혼합 정밀도

#### 🟢 선택 사항 (기본값으로 충분)
- **callbacks** (기본값: `None`)
  - 학습 과정에서 실행할 콜백 함수들의 리스트입니다.
  - 체크포인트 저장, 조기 종료 등을 위해 사용합니다.

- **logger** (기본값: `True`)
  - 학습 메트릭과 로그를 기록할 로거를 설정합니다.
  - 기본값 `True`는 CSVLogger를 사용합니다.

- **enable_checkpointing** (기본값: `True`)
  - 자동 체크포인트 저장 기능을 활성화합니다.

- **enable_progress_bar** (기본값: `True`)
  - 학습 진행 상황을 시각적으로 보여주는 진행률 표시줄을 활성화합니다.

- **enable_model_summary** (기본값: `True`)
  - 학습 시작 시 모델 구조와 파라미터 정보를 출력합니다.

- **log_every_n_steps** (기본값: `50`)
  - 지정된 스텝마다 로그를 기록합니다.
  - 너무 자주 로깅하면 성능에 영향을 줄 수 있습니다.

- **val_check_interval** (기본값: `1.0`)
  - 검증을 실행할 간격을 설정합니다.
  - `1.0`: 매 에포크마다 검증
  - `0.5`: 에포크의 절반마다 검증
  - `100`: 100 스텝마다 검증

- **check_val_every_n_epoch** (기본값: `1`)
  - N 에포크마다 검증을 실행합니다.

- **deterministic** (기본값: `False`)
  - 재현 가능한 결과를 위해 모든 연산을 결정적으로 실행합니다.
  - 성능이 약간 저하될 수 있지만, 실험 재현성을 보장합니다.

### 사용 예시

#### 최소 설정 (필수만)

```python
trainer = pl.Trainer(
    max_epochs=100,  # 유일한 필수 설정
)
```

#### 권장 설정

```python
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    devices="auto",
    precision="16-mixed",  # 메모리 절약
)
```

#### 완전한 설정 (모든 옵션 명시)

```
python
trainer = pl.Trainer(
    # 필수 설정
    max_epochs=args.max_epochs,

    # 권장 설정
    accelerator=args.accelerator,
    devices=args.devices,
    precision=args.precision,

    # 선택 사항
    callbacks=callbacks,
    logger=logger,
    enable_checkpointing=True,
    enable_progress_bar=True,
    enable_model_summary=True,
    log_every_n_steps=50,
    val_check_interval=1.0,
    check_val_every_n_epoch=1,
    deterministic=True,
)
```

### 매개변수 기본값 정리

| 파라미터                      | 기본값         | 중요도   | 설명                         |
|---------------------------|-------------|-------|----------------------------|
| `max_epochs`              | `None`      | 🔴 필수 | 무한 학습 방지를 위해 필수 설정         |
| `accelerator`             | `"auto"`    | 🟡 권장 | 하드웨어 자동 감지                 |
| `devices`                 | `"auto"`    | 🟡 권장 | 사용 가능한 모든 디바이스             |
| `precision`               | `"32-true"` | 🟡 권장 | 메모리 절약을 위해 `"16-mixed"` 권장 |
| `callbacks`               | `None`      | 🟢 선택 | 필요에 따라 추가                  |
| `logger`                  | `True`      | 🟢 선택 | 기본 CSVLogger 사용            |
| `enable_checkpointing`    | `True`      | 🟢 선택 | 체크포인트 자동 저장                |
| `enable_progress_bar`     | `True`      | 🟢 선택 | 진행률 표시                     |
| `enable_model_summary`    | `True`      | 🟢 선택 | 모델 요약 출력                   |
| `log_every_n_steps`       | `50`        | 🟢 선택 | 로깅 주기                      |
| `val_check_interval`      | `1.0`       | 🟢 선택 | 검증 간격                      |
| `check_val_every_n_epoch` | `1`         | 🟢 선택 | 검증 에포크 주기                  |
| `deterministic`           | `False`     | 🟢 선택 | 재현성 vs 성능 트레이드오프           |

In [7]:
from src.util.helper import fix_random_seed


fix_random_seed(SEED)

Seed set to 432


In [8]:
# transform 준비
from torchvision import transforms


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transform = transforms.Compose(
    [transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)

test_transform = transforms.Compose(
    [transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)

In [9]:
# DataModule 준비
from src.data.datamodules import DocumentImageDataModule


data_module = DocumentImageDataModule(
    batch_size=BATCH_SIZE,
    train_transform=train_transform,
    test_transform=test_transform,
    val_rate=VAL_RATE,
    random_seed=SEED,
    num_workers=NUM_WORKERS,
)

In [10]:
from torch.nn import CrossEntropyLoss

from src.data.datasets import DocumentImageSet


# Model 준비
criterion = CrossEntropyLoss()
resnet34_classifier = DocumentImageClassifier(
    model_name=MODEL_NAME,
    num_classes=DocumentImageSet.calculate_metadata_classes(),
    learning_rate=LEARNING_RATE,
    criterion=criterion,
)

In [11]:
# 콜백 준비
from src.training.callbacks import get_callbacks


callbacks = get_callbacks()

In [12]:
# trainer 준비
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

from src import config


logger = CSVLogger(
    save_dir=config.LOG_ROOT_DIR,
    name=f"train-log-of-{MODEL_NAME}",
)

trainer = Trainer(
    max_epochs=EPOCHS,
    callbacks=callbacks,
    logger=logger,
    log_every_n_steps=EPOCHS // 5,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [13]:
# 학습
trainer.fit(model=resnet34_classifier, datamodule=data_module)

[2025-07-06 19:19:16] INFO [DocumentImageDataModule.prepare_data] Done preparing data.
[2025-07-06 19:19:20] INFO [DocumentImageDataModule.setup] Done setup
/Users/joyuiyeong/.pyenv/versions/deeplearning/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:658: Checkpoint directory /Users/joyuiyeong/projects/upstageailab-cv-classification-cv_3/data/raw/checkpoints exists and is not empty.

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ResNet             | 21.3 M | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | train_f1       | MulticlassF1Score  | 0      | train
4 | val_accuracy   | MulticlassAccuracy | 0      | train
5 | val_f1         | MulticlassF1Score  | 0      | train
6 | test_accuracy  | MulticlassAccuracy | 0      | train
7 | test_f1        | MulticlassF1Score  | 0      | train
-------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 1.587
Epoch 0, global step 20: 'val_loss' reached 1.58677 (best 1.58677), saving model to '/Users/joyuiyeong/projects/upstageailab-cv-classification-cv_3/data/raw/checkpoints/epoch=00-val_loss=1.59.ckpt' as top 3


...

Epoch 7, global step 160: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 8, global step 180: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 9, global step 200: 'val_loss' was not in top 3
`Trainer.fit` stopped: `max_epochs=10` reached.
